In [44]:
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
import re
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd

driverPath = "chromedriver.exe" #填入剛剛記下的路徑
driver = webdriver.Chrome(driverPath) # Chrome
driver.implicitly_wait(30)
driver.get("https://www.yes123.com.tw/admin/index.asp")
search_input = driver.find_element_by_name("find_key1")
search_input.send_keys('富邦人壽')
start_search_btn = driver.find_element_by_class_name("bt_search")
start_search_btn.click()
condition = EC.visibility_of_element_located((By.CLASS_NAME,'sift'))
WebDriverWait(driver, 20, 0.5).until(condition)
soup = BeautifulSoup(driver.page_source,"html.parser")
s = re.findall("第.+頁",soup.text)[0]
totalpage = int(re.findall('\d+',s)[1])
current = int(re.findall('\d+',s)[0])

url_list=[]
for page in range(1,totalpage+1):
    print('page:'+str(page))
    while(current!=page):
        condition = EC.presence_of_element_located((By.NAME,"jump_page_num"))
        WebDriverWait(driver, 20, 0.5).until(condition)
        jump_page = driver.find_element_by_name("jump_page_num")
        jump_page.send_keys(page)
        
        condition = EC.element_to_be_clickable((By.CLASS_NAME,"bt_page"))
        WebDriverWait(driver, 20, 0.5).until(condition)
        jump_page_btn = driver.find_element_by_class_name("bt_page")
        jump_page_btn.click()
        
        condition = EC.visibility_of_element_located((By.CLASS_NAME,'sift'))
        WebDriverWait(driver, 20, 0.5).until(condition)
        soup = BeautifulSoup(driver.page_source,"html.parser")
        s = re.findall("第.+頁",soup.text)[0]
        current = int(re.findall('\d+',s)[0])

    for i in soup.find_all(class_='jobname'):
        if i.get('href'):
            url_list.append('https://www.yes123.com.tw/admin/'+i['href'])
            
            
result=[]
for url in url_list:
    print(url)
    driver.get(url)
    condition = EC.visibility_of_element_located((By.CLASS_NAME,"comp_detail"))
    WebDriverWait(driver, 20, 0.5).until(condition)
    soup = BeautifulSoup(driver.page_source,"html.parser")
    url_company ='https://www.yes123.com.tw/admin/job_refer_comp_info.asp?'+re.findall('p_id.+&',url)[0][:-1]
    driver.get(url_company)
    condition = EC.visibility_of_element_located((By.CLASS_NAME,'company_title'))
    WebDriverWait(driver, 20, 0.5).until(condition)
    soup_c = BeautifulSoup(driver.page_source,"html.parser")
    company = soup_c.find_all(class_='company_title')[0].text
    result.append([url,soup,company])
    sleep(0.3)

    
driver.close()

def getcontent(soup):
    soup_ul = soup.find_all('ul')
    sub_title = []
    sub_content = []
    for i in soup_ul:
        tt = i.find_all(class_="tt")
        rr = i.find_all(class_="rr")
        if tt!=[] and rr!=[]:
            for j in range(len(tt)):
                sub_title.append(tt[j].text.replace(' ： ',''))
                sub_content.append(rr[j].text.replace('\n','').replace('\t','').replace('\xa0','').replace('每月薪資行情表我要申訴','').replace(' ','').replace('地圖','').replace('就業導航',''))
    return dict(zip(sub_title,sub_content))

colname = ['web','url','工作名稱','薪資','上班地點','上班時段','公司','工作性質','聯絡人']
data=[]
for row in result:
    content = getcontent(row[1])
    value = ['yes123',
             row[0],
             row[1].find_all('h1')[0].text,
             content['薪資待遇'],
             content['工作地點'],
             content['上班時段'],
             row[2],
             content['工作性質'],
             content['連絡人'] ]
    r = dict(zip(colname,value))
    data.append(r)
df = pd.DataFrame(data)

page:1
page:2
page:3
page:4
https://www.yes123.com.tw/admin/job_refer_comp_job_detail2.asp?p_id=20180507174055_27935073&job_id=20180516170236_12955
https://www.yes123.com.tw/admin/job_refer_comp_job_detail2.asp?p_id=20180718182843_27935073&job_id=20180718183917_44846268
https://www.yes123.com.tw/admin/job_refer_comp_job_detail2.asp?p_id=20180116140808_27935073&job_id=20180117120511_15155959
https://www.yes123.com.tw/admin/job_refer_comp_job_detail2.asp?p_id=20190503103525_27935073&job_id=20190506144024_39042509
https://www.yes123.com.tw/admin/job_refer_comp_job_detail2.asp?p_id=20100805103727_27935073&job_id=20181225165844_396971
https://www.yes123.com.tw/admin/job_refer_comp_job_detail2.asp?p_id=20190225142316_27935073&job_id=20190228142628_19216637
https://www.yes123.com.tw/admin/job_refer_comp_job_detail2.asp?p_id=20181212172942_27935073&job_id=20190101132159_7855926
https://www.yes123.com.tw/admin/job_refer_comp_job_detail2.asp?p_id=20191025172905_27935073&job_id=20191115093603_104

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=81.0.4044.92)


In [30]:
df.shape

In [80]:
driver = webdriver.Chrome(driverPath) # Chrome
for url in url_list:
    print(url)
    driver.get(url)
    condition = EC.visibility_of_element_located((By.CLASS_NAME,"comp_detail"))
    WebDriverWait(driver, 20, 0.5).until(condition)
    soup = BeautifulSoup(driver.page_source,"html.parser")
    url_company ='https://www.yes123.com.tw/admin/job_refer_comp_info.asp?'+re.findall('p_id.+&',url)[0][:-1]
    driver.get(url_company)
    condition = EC.visibility_of_element_located((By.CLASS_NAME,'company_title'))
    driver.get(url_company)
    soup_c = BeautifulSoup(driver.page_source,"html.parser")
    company = soup_c.find_all(class_='company_title')[0].text
    result.append([url,soup,company])

https://www.yes123.com.tw/admin/job_refer_comp_job_detail2.asp?p_id=20180507174055_27935073&job_id=20180516170236_12955


IndexError: list index out of range

In [75]:
BeautifulSoup(driver.page_source,"html.parser")

<html xmlns="http://www.w3.org/1999/xhtml"><head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="no-cache" http-equiv="pragma"/>
<meta content="no-cache, must-revalidate" http-equiv="cache-control"/>
<meta content="0" http-equiv="expires"/>
<meta content="富邦人壽：安康展業處‧周美暖正在尋找 展業員NCT(正職)。更多 富邦人壽：安康展業處‧周美暖 人身保險 徵才、人才、招募資訊及職缺請上yes123求職網。" name="description"/>
<meta content="金融保險,富邦人壽保險股份有限公司,富邦人壽：安康展業處‧周美暖,yes123求職網,yes123,找工作,工作,求職,人力銀行" name="keywords"/>
<title>富邦人壽：安康展業處‧周美暖 公司簡介-yes123求職網</title>
<link href="css_2014/reset.css" rel="stylesheet" type="text/css"/>
<link href="css_2014/basic01_201608.css?v=5" rel="stylesheet" type="text/css"/>
<link href="css_2014/select.css" rel="stylesheet" type="text/css"/>
<link href="favicon.ico" rel="SHORTCUT ICON"/>
<link href="https://www.yes123.com.tw/admin/job_refer_comp_info.asp?p_id=20180507174055_27935073" rel="canonical"/>
<link href="https://m.yes123.com.tw/memmvc/jobSearch/EPIdc?pid=20180507174055_279350

In [77]:
driver.refresh()